In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
from divergen import CodebaseAssistant

# CodebaseAssistant

In [14]:
codebase_assitant = CodebaseAssistant(
    source_dir="../src/",
    prompt_manager={"prompt_library":"../assets/prompt-library"}
)

### Generating docstings

In [15]:
codebase_assitant.generate_docstrings(entity_name="PromptManager")

In [16]:
codebase_assitant.apply_changes()

In [17]:
codebase_assitant.revert_changes()

In [18]:
codebase_assitant.reject_changes()

### Explaining codebase

In [20]:
codebase_assitant.explain_codebase(
    entity_name="CodebaseAssistant", copy_to_clipboard=True
)

# CodebaseManager

The codebase assistant inherits from this class, however, we instantiate it on its own here for demo purpose

In [21]:
from divergen import CodebaseManager

In [22]:
codebase_manager = CodebaseManager(source_dir="../src")

In [23]:
codebase_manager.parse_modules()

### Getting specific entity (class, method, function)

In [24]:
print(
    codebase_manager.get_entity_source_code(entity_name="CodebaseAssistant")
)

class CodebaseAssistant(CodebaseManager):
    backup_dir: str = '.backup'
    prompt_manager: PromptManager
    _target_files: list = PrivateAttr(default_factory=list)
    _preview_files: list = PrivateAttr(default_factory=list)
    _backup_files: list = PrivateAttr(default_factory=list)

    def explain_codebase(self, entity_name: str=None, module_name: str=None, copy_to_clipboard: bool=True):
        self.reset_codebase()
        self.parse_modules()
        if entity_name is not None:
            source_code = self.get_entity_source_code(entity_name, module_name)
        elif module_name is not None:
            source_code = self.get_module_source_code(module_name)
        else:
            source_code = self.get_codebase_source_code()
        if copy_to_clipboard:
            prompt = self.prompt_manager.build_template(template_name='explain-codebase.yaml', template_inputs={'code': source_code})
            pyperclip.copy(prompt)
        else:
            return self.execute_gener

In [25]:
print(
    codebase_manager.get_entity_source_code(entity_name="CodebaseAssistant.explain_codebase")
)

def explain_codebase(self, entity_name: str=None, module_name: str=None, copy_to_clipboard: bool=True):
    self.reset_codebase()
    self.parse_modules()
    if entity_name is not None:
        source_code = self.get_entity_source_code(entity_name, module_name)
    elif module_name is not None:
        source_code = self.get_module_source_code(module_name)
    else:
        source_code = self.get_codebase_source_code()
    if copy_to_clipboard:
        prompt = self.prompt_manager.build_template(template_name='explain-codebase.yaml', template_inputs={'code': source_code})
        pyperclip.copy(prompt)
    else:
        return self.execute_generate_docstring_template(template_inputs={'code': source_code})


### Getting the source code of a module

In [26]:
print(
    codebase_manager.get_module_source_code(module_name="codebase_manager.py")
)

import glob
import os
from typing import Dict
from pydantic import BaseModel, PrivateAttr
from divergen.codebase_manager.module_parser import ModuleParser

class CodebaseManager(BaseModel):
    source_dir: str
    _modules: Dict[str, ModuleParser] = PrivateAttr(default_factory=dict)

    def get_codebase_source_code(self):
        codebase_source_code = ''
        for module_path, module_content in self._modules.items():
            codebase_source_code += f'\n\n===========================\n' + f'Code for module {module_path}:\n' + '===========================\n\n' + f'{module_content.source_code}'
        return codebase_source_code

    def get_module_source_code(self, module_name):
        module_path = self.get_module_path(module_name)
        module_content = self._modules[module_path]
        return module_content.source_code

    def get_entity_source_code(self, entity_name, module_name=None):
        if module_name is not None:
            module_path = self.get_module_path(mod

In [27]:
print(
    codebase_manager.get_codebase_source_code()
)



Code for module ../src/divergen/codebase_manager/codebase_manager.py:

import glob
import os
from typing import Dict
from pydantic import BaseModel, PrivateAttr
from divergen.codebase_manager.module_parser import ModuleParser

class CodebaseManager(BaseModel):
    source_dir: str
    _modules: Dict[str, ModuleParser] = PrivateAttr(default_factory=dict)

    def get_codebase_source_code(self):
        codebase_source_code = ''
        for module_path, module_content in self._modules.items():
            codebase_source_code += f'\n\n===========================\n' + f'Code for module {module_path}:\n' + '===========================\n\n' + f'{module_content.source_code}'
        return codebase_source_code

    def get_module_source_code(self, module_name):
        module_path = self.get_module_path(module_name)
        module_content = self._modules[module_path]
        return module_content.source_code

    def get_entity_source_code(self, entity_name, module_name=None):
        if mo

In [8]:
import pyperclip
pyperclip.copy(codebase_manager.get_codebase_source_code())